# K-NEAREST-NEIGHBOUR CLASSIFIER

In [744]:
import os
import json
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [745]:
df = pd.read_json("cleaned_train.json")

In [746]:
features = [
    'bathrooms', 
    'bedrooms', 
    'latitude', 
    'longitude', 
    'price',
    'Special Characters',
    'Numbers', 
    'StopWord',
    'Uppercase',
]
df_X = df[features]
df_X

,bathrooms,bedrooms,latitude,longitude,price,Special Characters,Numbers,StopWord,Uppercase
4,1.0,1,40.7108,-73.9539,2400,20,2,8,45
6,1.0,2,40.7513,-73.9722,3800,9,2,40,71
9,1.0,2,40.7575,-73.9625,3495,11,2,28,62
10,1.5,3,40.7145,-73.9425,3000,8,1,8,61
15,1.0,0,40.7439,-73.9743,2795,6,1,0,36
...,...,...,...,...,...,...,...,...,...
123998,1.0,0,40.7640,-73.9917,2175,0,0,17,7
124000,1.0,3,40.8433,-73.9396,2800,3,1,38,16
124002,1.0,2,40.8198,-73.9578,2395,3,1,16,12
124008,1.0,2,40.7448,-74.0017,4195,9,1,29,25


In [747]:
#Classification label
df_y = df['interest_level']

In [748]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import f1_score

In [749]:
X_train,X_test,y_train,y_test = train_test_split(df_X,df_y)

In [750]:
from sklearn.neighbors import KNeighborsClassifier

In [751]:
knn = KNeighborsClassifier()

In [752]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [753]:
y_pred = knn.predict(X_test)

In [754]:
print("Accuracy", metrics.accuracy_score(y_test,y_pred))

Accuracy 0.6579398091509665


In [755]:
scores_knn = cross_val_score(knn,df_X,df_y,cv=5,n_jobs=4)

In [756]:
scores_knn

array([0.66418595, 0.66924959, 0.65303834, 0.65558728, 0.65497553])

In [757]:
scores_knn.mean()

0.6594073371543295

In [758]:
f1_score(y_test,y_pred,average="micro")

0.6579398091509665

In [759]:
#test.json COMES STRAIGHT FROM KAGGLE WEBSITE
#need unformatted to access the listing ID's
test_df = pd.read_json("test2.json")
test_df_unformatted = pd.read_json("test.json")

In [768]:
knn_pred = knn.predict_proba(test_df)
knn_pred

,bathrooms,bedrooms,latitude,longitude,price,Special Characters,Numbers,StopWord,Uppercase
0,1.0,1,40.7185,-73.9865,2950,8,0,19,37
1,1.0,2,40.7278,-74.0000,2850,2,0,5,15
2,1.0,0,40.7260,-74.0026,2295,0,0,10,9
3,1.0,2,40.7321,-74.0028,2900,2,0,3,5
5,1.0,1,40.7054,-74.0095,3254,7,1,19,36
...,...,...,...,...,...,...,...,...,...
124003,1.0,1,40.7925,-73.9454,1700,0,1,0,193
124005,1.0,2,40.7456,-73.9797,4195,18,0,30,12
124006,1.0,0,40.7416,-73.9829,2400,1,0,52,16
124007,2.0,2,40.7485,-73.9800,6895,7,0,10,153


In [761]:
p1 = [near[0] for near in knn_pred]
p2 = [near[1] for near in knn_pred]
p3 = [near[2] for near in knn_pred]
answer_knn = pd.DataFrame({
    "listing_id": test_df_unformatted['listing_id'],
    "high": p1,
    "medium": p2,
    "low": p3
})

In [762]:
answer_knn.to_csv("knn.csv",index=False)

# Normalizing the data and running KNN on it

In [763]:
from sklearn import preprocessing

In [769]:
#normalizing the cleaned training dataset
x = df_X.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X_normalized = pd.DataFrame(x_scaled)

In [770]:
X_trainnorm,X_testnorm,y_trainnorm,y_testnorm = train_test_split(df_X_normalized,df_y)

In [872]:
knnNorm = KNeighborsClassifier(n_neighbors=6)

In [873]:
knnNorm.fit(X_trainnorm,y_trainnorm)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [874]:
y_prednorm = knnNorm.predict(X_testnorm)
y_prednorm

array(['high', 'low', 'low', ..., 'low', 'low', 'medium'], dtype=object)

In [875]:
print("Accuracy", metrics.accuracy_score(y_testnorm,y_prednorm))

Accuracy 0.6822445151292716


In [876]:
scores_knnNorm = cross_val_score(knnNorm,df_X_normalized,df_y,cv=5,n_jobs=4)
scores_knnNorm

array([0.67845856, 0.67526509, 0.68597064, 0.670677  , 0.66945351])

In [877]:
scores_knnNorm.mean()

0.6759649580228698

In [878]:
f1_score(y_testnorm,y_prednorm,average="micro")

0.6822445151292716

In [879]:
#normalizing the test data
x = test_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_test_X_normalized = pd.DataFrame(x_scaled)

In [880]:
knnNorm_pred = knnNorm.predict_proba(df_test_X_normalized)

In [881]:
p1 = [near[0] for near in knnNorm_pred]
p2 = [near[1] for near in knnNorm_pred]
p3 = [near[2] for near in knnNorm_pred]
answer_knnNorm = pd.DataFrame({
    "listing_id": test_df_unformatted['listing_id'],
    "high": p1,
    "medium": p2,
    "low": p3
})

In [882]:
answer_knnNorm.to_csv("knnNorm.csv",index=False)

# Reduce dimensionality of dataset with normalized values, and tweaking the hyperparameter of KNN

In [440]:
df_X.corr()

,bathrooms,bedrooms,latitude,longitude,price,Special Characters,Numbers,StopWord,Uppercase
bathrooms,1.000000,0.524790,0.014507,-0.016609,0.685864,0.102983,0.171477,0.072079,0.074406
bedrooms,0.524790,1.000000,-0.000255,0.009252,0.538388,0.093101,0.231826,0.035671,0.126637
latitude,0.014507,-0.000255,1.000000,-0.103803,-0.026881,-0.021422,-0.030193,-0.030825,0.005373
longitude,-0.016609,0.009252,-0.103803,1.000000,-0.081731,-0.012763,0.009180,-0.012177,-0.019809
price,0.685864,0.538388,-0.026881,-0.081731,1.000000,0.111628,0.128479,0.083365,0.061489
Special Characters,0.102983,0.093101,-0.021422,-0.012763,0.111628,1.000000,0.220135,0.160523,0.289114
Numbers,0.171477,0.231826,-0.030193,0.009180,0.128479,0.220135,1.000000,0.286622,0.140193
StopWord,0.072079,0.035671,-0.030825,-0.012177,0.083365,0.160523,0.286622,1.000000,-0.103926
Uppercase,0.074406,0.126637,0.005373,-0.019809,0.061489,0.289114,0.140193,-0.103926,1.000000


In [1024]:
features_new = [
    "bathrooms",
    "price",
    "Numbers"
]
df_X_new = df[features_new]
df_X_new

,bathrooms,price,Numbers
4,1.0,2400,2
6,1.0,3800,2
9,1.0,3495,2
10,1.5,3000,1
15,1.0,2795,1
...,...,...,...
123998,1.0,2175,0
124000,1.0,2800,1
124002,1.0,2395,1
124008,1.0,4195,1


In [1025]:
df_y = df['interest_level']

In [1026]:
#normalizing the cleaned training dataset of reduced features
x = df_X_new.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X_normalized = pd.DataFrame(x_scaled)

In [1063]:
knnRed = KNeighborsClassifier(n_neighbors=10)

In [1064]:
X_train,X_test,y_train,y_test = train_test_split(df_X_normalized,df_y)

In [1065]:
knnRed.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [1066]:
y_pred = knnRed.predict(X_test)
y_pred

array(['low', 'low', 'low', ..., 'low', 'low', 'high'], dtype=object)

In [1067]:
print("Accuracy",metrics.accuracy_score(y_test,y_pred))

Accuracy 0.6867302830111737


In [1068]:
scores_knnRed = cross_val_score(knnRed,df_X_normalized,df_y,cv=5,n_jobs=4)
scores_knnRed

array([0.67641961, 0.68352365, 0.67801794, 0.67414356, 0.67557096])

In [1069]:
scores_knnRed.mean()

0.6775351464189511

In [1070]:
f1_score(y_test,y_pred,average="micro")

0.6867302830111737

In [1071]:
test_df_new = test_df[features_new]

In [1072]:
#normalizing the test data
x = test_df_new.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_test_X_normalized = pd.DataFrame(x_scaled)

In [1073]:
knnRedpred = knnRed.predict_proba(df_test_X_normalized)

In [1074]:
p1 = [red[0] for red in knnRedpred]
p2 = [red[1] for red in knnRedpred]
p3 = [red[2] for red in knnRedpred]
answer_knnRed = pd.DataFrame({
    "listing_id": test_df_unformatted['listing_id'],
    "high": p1,
    "medium": p2,
    "low": p3
})
answer_knnRed.to_csv("knnRed.csv",index=False)